In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports

In [92]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [93]:
from dataloading import get_labeled_data

# Load the data

In [94]:
# file paths
train_path = "../data/preprocessed/augmented_vectors.csv"
scaled_train_path = "../data/preprocessed/train_vectors_scaled.csv"
valid_path = "../data/preprocessed/validation_vectors.csv"
scaled_valid_path = "../data/preprocessed/validation_vectors_scaled.csv"

In [95]:
# load the dataframes
train_df = pd.read_csv(train_path)
train_df_scaled = pd.read_csv(scaled_train_path)
valid_df = pd.read_csv(valid_path)
valid_df_scaled = pd.read_csv(scaled_valid_path)

In [96]:
# using the scaled dataframe
train_df_scaled.head()

,Port number,Packet speed,Data speed,Avg packet len,Source IP count,victim IP num,time_of_day,Significant flag,CLDAP,Generic UDP,...,IPv4 fragmentation,Suspicious traffic,SSDP,TCP Anomaly,SNMP,DNS,is_weekday,Type,is_synthetic,other_attack_codes
0,-0.817317,-0.258647,-0.114203,0.752471,-0.179160,-0.604654,0.597073,0,0,0,...,0,0,0,0,0,0,1,Normal traffic,False,0
1,-0.817317,-0.195674,0.024471,0.978049,-0.191742,-0.604474,0.670414,0,0,0,...,0,0,0,0,0,0,1,Normal traffic,False,0
2,-0.941431,-0.225965,-0.065259,0.781814,-0.191742,-0.604294,0.680810,0,0,0,...,0,0,0,0,0,0,1,Normal traffic,False,0
3,-0.941431,-0.178138,-0.008159,0.781814,-0.191742,-0.604294,0.680854,0,0,0,...,0,0,0,0,0,0,1,Normal traffic,False,0
4,-0.817317,-0.227559,-0.016316,0.941370,-0.191742,-0.604474,0.697654,0,0,0,...,0,0,0,0,0,0,1,Normal traffic,False,0


In [97]:
# get target features
train_df_scaled = train_df_scaled[train_df_scaled['is_synthetic'] == 0]
data_df = train_df_scaled.drop(columns=['is_synthetic','Type'])
target_df = train_df_scaled['Type']

# encode target labels
target_label_encoder = LabelEncoder()
target_label_encoder.fit(target_df)
target_df = target_label_encoder.transform(target_df)

X = data_df.to_numpy()
y = target_df

In [98]:
# train - test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [99]:
# Initialize RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Predict on the testing data
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9492708537531216


In [100]:
from sklearn.metrics import classification_report

target_names = target_label_encoder.inverse_transform([0,1,2])

print(classification_report(y_test, y_pred, target_names=target_names))

                    precision    recall  f1-score   support

       DDoS attack       0.97      0.99      0.98     15796
    Normal traffic       0.95      0.99      0.97     89883
Suspicious traffic       0.86      0.59      0.70     11650

          accuracy                           0.95    117329
         macro avg       0.93      0.85      0.88    117329
      weighted avg       0.95      0.95      0.94    117329



In [101]:
# get target features
data_df = valid_df_scaled.drop(columns=['Type'])
target_df = valid_df_scaled['Type']

# encode target labels
target_df = target_label_encoder.transform(target_df)

X_valid = data_df.to_numpy()
y_valid = target_df

In [102]:
y_valid.shape

(1247266,)

In [103]:
y_pred = rf_classifier.predict(X_valid)

# Evaluate the model
accuracy = accuracy_score(y_valid, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.24462785003359347


In [104]:
print(classification_report(y_valid, y_pred, target_names=target_names))

c:\Users\Maya\.conda\envs\ddos\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Maya\.conda\envs\ddos\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                    precision    recall  f1-score   support

       DDoS attack       0.24      1.00      0.39    305167
    Normal traffic       0.00      0.00      0.00    919196
Suspicious traffic       0.00      0.00      0.00     22903

          accuracy                           0.24   1247266
         macro avg       0.08      0.33      0.13   1247266
      weighted avg       0.06      0.24      0.10   1247266



c:\Users\Maya\.conda\envs\ddos\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
